In [ ]:
#Proviamo a farle in v2

#from tensorflow import keras
#from minio import Minio
#import numpy as np
#import json
import kfp
from kfp import dsl
import os
from kfp.dsl import Input, Output, Dataset, Model, Metrics, ClassificationMetrics, Artifact


 

@dsl.component(base_image="tensorflow/tensorflow")
def load_dataset(x_train_artifact: Output[Dataset], x_test_artifact: Output[Dataset],y_train_artifact: Output[Dataset],y_test_artifact: Output[Dataset]):
    '''
    get dataset from Keras and load it separating input from output and train from test
    '''
    import numpy as np
    from tensorflow import keras
    import os
    
   
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    np.save("/tmp/x_train.npy",x_train)
    os.rename("/tmp/x_train.npy", x_train_artifact.path)
    
    np.save("/tmp/y_train.npy",y_train)
    os.rename("/tmp/y_train.npy", y_train_artifact.path)
    
    np.save("/tmp/x_test.npy",x_test)
    os.rename("/tmp/x_test.npy", x_test_artifact.path)
    
    np.save("/tmp/y_test.npy",y_test)
    os.rename("/tmp/y_test.npy", y_test_artifact.path)

    print(f"x_train shape: {x_train.shape}")
    print(f"y_train shape: {y_train.shape}")

    print(f"x_test shape: {x_test.shape}")
    print(f"y_test shape: {y_test.shape}")
    
    print("Success!")

@dsl.component(packages_to_install=['numpy'])
def preprocessing(metrics : Output[Metrics], x_train_processed : Output[Dataset], x_test_processed: Output[Dataset],
                  x_train_artifact: Input[Dataset], x_test_artifact: Input[Dataset]):
    ''' 
    just reshape and normalize data
    '''
    import numpy as np
    import os
    
    # load data artifact store
    x_train = np.load(x_train_artifact.path) 
    x_test = np.load(x_test_artifact.path)
    
    # reshaping the data
    # reshaping pixels in a 28x28px image with greyscale, canal = 1. This is needed for the Keras API
    x_train = x_train.reshape(-1,28,28,1)
    x_test = x_test.reshape(-1,28,28,1)
    # normalizing the data
    # each pixel has a value between 0-255. Here we divide by 255, to get values from 0-1
    x_train = x_train / 255
    x_test = x_test / 255
    
    #logging metrics using Kubeflow Artifacts
    metrics.log_metric("Len x_train", x_train.shape[0])
    metrics.log_metric("Len y_train", x_test.shape[0])
   
    
    # save feuture in artifact store
    np.save("tmp/x_train.npy",x_train)
    os.rename("tmp/x_train.npy", x_train_processed.path)
    
    np.save("tmp/x_test.npy",x_test)
    os.rename("tmp/x_test.npy", x_test_processed.path)
    

@dsl.component(base_image="tensorflow/tensorflow")
def model_building(ml_model : Output[Model]):
    '''
    Define the model and load it (not yet compiled to minio)
    This way it's more simple to change the model architecture and all the steps and indipendent
    '''
    from tensorflow import keras
    import tensorflow as tf
    import os
    
    #model definition
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28,28,1)))
    model.add(keras.layers.MaxPool2D(2, 2))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(32, activation='relu'))

    model.add(keras.layers.Dense(10, activation='softmax'))
    
    summary = model.summary()
    
    #saving model
    model.save(ml_model.path)
    #os.rename("/tmp/model.keras", ml_model.path)
   
    print("Success")
    
@dsl.component(base_image="tensorflow/tensorflow", packages_to_install=['scikit-learn'])
def model_training(
    ml_model : Input[Model],
    x_train_processed : Input[Dataset], x_test_processed: Input[Dataset],
    y_train_artifact : Input[Dataset], y_test_artifact :Input[Dataset],
    hyperparameters : dict, 
    metrics: Output[Metrics], classification_metrics: Output[ClassificationMetrics], model_trained: Output[Model]
    ):
    """
    Build the model with Keras API
    Export model metrics
    """
    from tensorflow import keras
    import tensorflow as tf
    import numpy as np
    import os
    import glob
    from sklearn.metrics import confusion_matrix
    
    #load dataset
    x_train = np.load(x_train_processed.path)
    x_test = np.load(x_test_processed.path)
    y_train = np.load(y_train_artifact.path)
    y_test = np.load(y_test_artifact.path)
    
    #load model structure
    model = keras.models.load_model(ml_model.path)
    
    #reading best hyperparameters from katib
    lr=float(hyperparameters["lr"])
    no_epochs = int(hyperparameters["num_epochs"])
    
    #compile the model - we want to have a binary outcome
    model.compile(tf.keras.optimizers.SGD(learning_rate=lr),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

    
    #fit the model and return the history while training
    history = model.fit(
      x=x_train,
      y=y_train,
      epochs=no_epochs,
      batch_size=20,
    )

     
    # Test the model against the test dataset
    # Returns the loss value & metrics values for the model in test mode.
    model_loss, model_accuracy = model.evaluate(x=x_test,y=y_test)
    
    #build a confusione matrix
    y_predict = model.predict(x=x_test)
    y_predict = np.argmax(y_predict, axis=1)
    cmatrix = confusion_matrix(y_test, y_predict)
    cmatrix = cmatrix.tolist()
    numbers_list = ['0','1','2','3','4','5','6','7','8','9']
    #log confusione matrix
    classification_metrics.log_confusion_matrix(numbers_list,cmatrix)
  
    #Kubeflox metrics export
    metrics.log_metric("Test loss", model_loss)
    metrics.log_metric("Test accuracy", model_accuracy)
    
    #adding /1/ subfolder for TFServing and saving model to artifact store
    model_trained.uri = model_trained.uri + '/1/'
    keras.models.save_model(model,model_trained.path)
    
@dsl.component(packages_to_install=['kserve','kubernetes'])
def model_serving(model_trained : Input[Model]):
    """
    Create kserve instance
    """
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1TFServingSpec
    from datetime import datetime
    import time
    import os
    
    #get model uri
    uri = model_trained.uri
    #replace minio with s3
    uri = uri.replace("minio","s3")
    #removing last subfolder /1/
    uri = uri.rsplit("/",2)[0]
    
    #TFServing wants this type of structure ./models/1/model
    # the number represent the model version
    # in this example we use only 1 version
    
    #Inference server config
    namespace = utils.get_default_target_namespace()
    now = datetime.now()
    name="digits-recognizer"
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version

    isvc = V1beta1InferenceService(api_version=api_version,
                                   kind=constants.KSERVE_KIND,
                                   metadata=client.V1ObjectMeta(
                                       name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                   spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       service_account_name="sa-minio-kserve",
                                       tensorflow=(V1beta1TFServingSpec(
                                           storage_uri=uri))))
    )

    KServe = KServeClient()
    
    #replace old inference service with a new one
    try:
        KServe.delete(name=name, namespace=namespace)
        print("Modello precedente eliminato")
    except:
        print("Non posso eliminare")
    time.sleep(10)
    
    KServe.create(isvc)

 
@dsl.pipeline(
    name='tutorial-dev',
    description='Detect digits')
def mnist_pipeline(hyperparameters: dict):
    load_task = load_dataset()
    preprocess_task = preprocessing(
        x_train_artifact = load_task.outputs["x_train_artifact"],
        x_test_artifact = load_task.outputs["x_test_artifact"]
    )
    model_building_task = model_building()
    training_task = model_training(
        ml_model = model_building_task.outputs["ml_model"],
        x_train_processed = preprocess_task.outputs["x_train_processed"],
        x_test_processed = preprocess_task.outputs["x_test_processed"],
        y_train_artifact = load_task.outputs["y_train_artifact"],
        y_test_artifact = load_task.outputs["y_test_artifact"],
        hyperparameters = hyperparameters
    )
    serving_task = model_serving(model_trained = training_task.outputs["model_trained"])
    
 #reading best hyperparameters from katib



with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

hyperparameters ={"hyperparameters" :  {"lr":0.1, "num_epochs":1 } }
client = kfp.Client(
    existing_token=TOKEN,
    host='http://ml-pipeline.kubeflow.svc.cluster.local:8888',
)
namespace="kubeflow-user-example-com"

#client.upload_pipeline_version(pipeline_package_path='./mnist-pipeline.yaml',pipeline_name="medium tutorial",pipeline_version_name= "1.0")
kfp.compiler.Compiler().compile(pipeline_func=mnist_pipeline,package_path='./mnist-classifier.yaml')
#client.upload_pipeline(pipeline_package_path='./mnist-classifier.yaml',pipeline_name="MNIST classifier tutorial",namespace = namespace)


client.create_run_from_pipeline_func(mnist_pipeline, arguments=hyperparameters,experiment_name="test",namespace="kubeflow-user-example-com",enable_caching=True)